<h1> Set up Azure workspaces </h1>

In [1]:
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.16.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

AI-For-Earh loaded


In [3]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'eec2f4e3-2641-43c4-a108-eaa29422f3cc'
resource_group = 'Mahesh-Resouce-Group'
workspace_name = 'AI-For-Earh'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='PreProAIFE')
maindf = dataset.to_pandas_dataframe()

<h1> Dataset operations </h1>

In [5]:
maindf.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,max_temp,sun_hour,dew_point,cloud_cover,precp,wind_speed,fire_or_not,elevation,1,2,...,20,21,22,23,24,25,26,27,28,current_date
1,38.0,12.8,14.0,13.0,0.4,11.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-15
2,38.0,12.8,14.0,13.0,0.4,11.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-15
3,40.0,12.8,13.0,5.0,0.0,7.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-16
4,40.0,12.8,13.0,5.0,0.0,7.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-16


In [4]:
maindf.columns = maindf.iloc[0]


In [5]:
maindf = maindf.iloc[1:,:]
maindf = maindf.reset_index(drop=True)

In [6]:
maindf.head()

,max_temp,sun_hour,dew_point,cloud_cover,precp,wind_speed,fire_or_not,elevation,1,2,...,20,21,22,23,24,25,26,27,28,current_date
0,38.0,12.8,14.0,13.0,0.4,11.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-15
1,38.0,12.8,14.0,13.0,0.4,11.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-15
2,40.0,12.8,13.0,5.0,0.0,7.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-16
3,40.0,12.8,13.0,5.0,0.0,7.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-16
4,42.0,12.8,8.0,3.0,0.0,9.0,0.0,264,0,0,...,0,0,0,0,0,0,0,0,0,2018-04-17


<h1> Convert datetime </h1>

In [8]:
maindf['current_date'] = pd.to_datetime(maindf['current_date'], 
 format = '%Y-%m-%d', 
 errors = 'coerce')

In [9]:
maindf['Day'] = maindf['current_date'].dt.day
maindf['Month'] = maindf['current_date'].dt.month
maindf['Year'] = maindf['current_date'].dt.year

<h1> Modelling </h1>

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import datetime as dt
from datetime import datetime


In [10]:
Y = maindf.fire_or_not

In [11]:
X = maindf.copy()

In [13]:
len(X)

11381351

In [12]:
X = X.drop(['fire_or_not','current_date'], axis=1)

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [17]:
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_score,recall_score,f1_score

In [22]:
classifiers = [
    LogisticRegression(),
    SVC(kernel = "rbf", gamma=2, C=1),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
   ]

In [23]:
model2 = SVC(kernel = "rbf", gamma=2, C=1)
model2.fit(X_train, Y_train)
score = model2.score(X_test, Y_test)
print('RBF SVM: ',score)

RBF SVM:  0.8906103561387066


In [17]:
model3 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
model3.fit(X_train, Y_train)
score = model3.score(X_test, Y_test)
print('RF: ',score)

RF:  0.8246595017350006


In [24]:
model1 = LogisticRegression()
model1.fit(X_train, Y_train)
score = model1.score(X_test, Y_test)
print('Logistic Regression: ',score)

Logistic Regression:  0.8910496719775071


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
